# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
# Import Dependencies.

from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

import logging
import os

import azureml.core
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from sklearn.model_selection import train_test_split

import pandas as pd
from azureml.widgets import RunDetails

import joblib 

from azureml.core.model import Model

from azureml.pipeline.core import PipelineData, TrainingOutput

from azureml.pipeline.core import Pipeline

import pickle

from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.webservice import LocalWebservice, AciWebservice

import requests
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview

There are some factors that affects Death Event. We use Kaggle's dataset Heart Failure Clinical Records which contains  information like age, sex, blood pressure, smoke, diabetes, ejection fraction, creatinine phosphokinase, serum_creatinine, serum_sodium, time and we have to predict their death_event.

Dataset was uploaded and register as dataset in the workspace


In [2]:
# Create a workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AP3VWNK52 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-131542
aml-quickstarts-131542
southcentralus
d7f39349-a66b-446e-aba6-0053c2cf1c11


In [3]:
# choose a name for experiment
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)

experiment



Name,Workspace,Report Page,Docs Page
ml-experiment-1,quick-starts-ws-131542,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster

amlcompute_cluster_name = "notebook131542"

# Verify that cluster does not exist already

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# For a more detailed view of current AmlCompute status, use get_status().


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
dataset = Dataset.get_by_name(workspace, name='heart')
ds = dataset.to_pandas_dataframe()

In [5]:
#upload Heart Failure Dataset

# data = pd.read_csv("heart_failure_clinical_records_dataset.csv")
# data.describe()


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [6]:
# train, test = train_test_split(data, test_size=0.2, random_state=1 )

if "training" not in os.listdir():
    os.mkdir("./training")

if "models" not in os.listdir():
    os.mkdir("./models")
    
# train.to_csv("training/train_data.csv", index = False)

# data_store = ws.get_default_datastore()
# data_store.upload(src_dir = "./training", target_path = 'udacity-project', overwrite = True,  show_progress = True)


Uploading an estimated of 1 files
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_0a5662e7ebd143b9911d22c1bfaa9389

In [7]:
# train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("udacity-project/train_data.csv"))
# train=TabularDatasetFactory.from_delimited_files([(datastore, 'trainset.csv')])


In [10]:
# test.to_csv(path_or_buf='testset.csv', index=False)
# data_store.upload_files(['testset.csv'])

# from azureml.data.dataset_factory import TabularDatasetFactory

# test_data = TabularDatasetFactory.from_delimited_files([(data_store, 'testset.csv')])


Uploading an estimated of 1 files
Target already exists. Skipping upload for testset.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

We are trying to predict DEATH_EVENT, therefore we need to define the task as classification. Also define Featurization as auto and enable early stopping and the primary metric AUC_weighted

In [12]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'    
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    
    compute_target = compute_target,
    task = "classification",
    training_data = ds,
    label_column_name = "DEATH_EVENT",   
    path = project_folder,
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    **automl_settings 
)

In [14]:
# TODO: Submit your experiment

# Create Pipeline and AutoMLStep / outputs AutoMLStep, TrainingOutput

# metrics_output_name = 'metrics_output'
# best_model_output_name = 'best_model_output'

# metrics_data = PipelineData(name='metrics_data',
#                           datastore=data_store,
#                           pipeline_output_name=metrics_output_name,
#                           training_output=TrainingOutput(type='Metrics'))
# model_data = PipelineData(name='model_data',
#                           datastore=data_store,
#                           pipeline_output_name=best_model_output_name,
#                           training_output=TrainingOutput(type='Model'))


remote_run = experiment.submit(automl_config, show_output = True)
RunDetails(remote_run).show()

#Create an AutoMLStep

# from azureml.pipeline.steps import AutoMLStep

# automl_step = AutoMLStep(
#     name='automl_module',
#     automl_config=automl_config,
#     outputs=[metrics_data, model_data],
#     allow_reuse=True)

# pipeline = Pipeline(
#     description="pipeline_with_automlstep",
#     workspace=ws,    
#     steps=[automl_step])


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
# pipeline_run = experiment.submit(pipeline, show_output=True)
remote_run.wait_for_completion(show_output=True)

Created step automl_module [6ff9df1c][c0cb7e0f-e61c-4ff0-8b40-c077482c09c4], (This step will run and generate new outputs)
Submitted PipelineRun 7f468927-4420-4cd6-aff1-1619b0393a9b
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-1/runs/7f468927-4420-4cd6-aff1-1619b0393a9b?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-131542/workspaces/quick-starts-ws-131542


In [16]:
# RunDetails(pipeline_run).show()
#Take SS

RunDetails(remote_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [17]:
# pipeline_run.wait_for_completion()

# ULIZEUZ: Remove
# remote_run.wait_for_completion()

This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


PipelineRunId: 7f468927-4420-4cd6-aff1-1619b0393a9b
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-1/runs/7f468927-4420-4cd6-aff1-1619b0393a9b?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-131542/workspaces/quick-starts-ws-131542
PipelineRun Status: Running




## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [27]:
best_model, fitted_model = remote_run.get_output()

print(best_model)

# Take SS

# Retrieve metrics of all child runs

# metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
# num_file_downloaded = metrics_output.download('.', show_progress=True)


In [ ]:
print(fitted_model)

In [ ]:
fitted_model.step

In [28]:
# import json
# with open(metrics_output._path_on_datastore) as f:
#     metrics_output_result = f.read()
#     
# deserialized_metrics_output = json.loads(metrics_output_result)
# df = pd.DataFrame(deserialized_metrics_output)
# df

# Retrieve and save your best automl model.

# best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
# num_file_downloaded = best_model_output.download('.', show_progress=True)

# with open(best_model_output._path_on_datastore, "rb" ) as f:
#     best_model = pickle.load(f)
# best_model


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  max_leaf_nodes=None,
                                                                                                  max_samples=None,
                                                                                                  min_impurity_decrease=0.0,
                                

In [29]:
#TODO: Save the best model
joblib.dump(fitted_model, "models/automl_hearth.pkl")


[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('61',
                                             Pipeline(memory=None,
                                                      steps=[('minmaxscaler',
                                                              MinMaxScaler(copy=True,
                                                                           feature_range=(0,
                                                                                          1))),
                                                         

['models/automl_hearth.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [33]:
model = Model.register(model_path = "./models/automl_hearth.pkl",
                       model_name = "automl_hearth.pkl",
                       description = "Best model trained with AutoML",
                       workspace = ws)


ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path description in the set of files uploaded to the run: ['logs/azureml/executionlogs.txt', 'logs/azureml/stderrlogs.txt', 'logs/azureml/stdoutlogs.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path description in the set of files uploaded to the run: ['logs/azureml/executionlogs.txt', 'logs/azureml/stderrlogs.txt', 'logs/azureml/stdoutlogs.txt']\n                See https://aka.ms/run-logging for more details."
    }
}

Registering model Best_Model


In [37]:
# Create the environment
myenv = Environment(workspace=ws, name="myenv")
# conda_dep = CondaDependencies()

## Define the packages needed by the model and scripts
# myenv.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
# myenv.python.conda_dependencies.add_pip_package("joblib")

# myenv.docker.base_image = None
# myenv.docker.base_dockerfile = "FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04\nRUN echo \"this is test\""
# myenv.inferencing_stack_version = "latest"

## conda_dep.add_conda_package("tensorflow")
## conda_dep.add_conda_package("numpy")
## conda_dep.add_conda_package("scikit-learn")
## You must list azureml-defaults as a pip dependency
## conda_dep.add_pip_package("azureml-defaults")
## conda_dep.add_pip_package("keras")
## conda_dep.add_pip_package("gensim")

## Adds dependencies to PythonSection of myenv
## myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)


In [41]:
# Define Deployment

deployment_config = AciWebservice.deploy_configuration()

# model = Model(ws, name='hearth')
service = Model.deploy(ws, 'myservice', [model], inference_config, deployment_config)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry ce398b1861e2422fa9d81168d43a807d.azurecr.io
Logging into Docker registry ce398b1861e2422fa9d81168d43a807d.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM ce398b1861e2422fa9d81168d43a807d.azurecr.io/azureml/azureml_2156255e96ba2ed1c4af8bd1a9412df9
 ---> 77201508465e
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 2a5bdc1dea44
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImQ3ZjM5MzQ5LWE2NmItNDQ2ZS1hYmE2LTAwNTNjMmNmMWMxMSIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzMTU0MiIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzMTU0MiIsIndvcmtzcGFjZUlkIjoiY2UzOThiMTgtNjFlMi00MjJmLWE5ZDgtMTE2OGQ0M2E4MDdkIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in c99b313f1f49
 ---> c7bac3ee4445
Step 4/5 : RUN mv '/var/azureml-app/tmpsza8lxd3.py' /var/azureml-app/main.py


In [42]:

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)


ERROR:azureml._model_management._util:Error: Container has crashed. Did your init method fail?



Checking container health...

Container Logs:
2020-12-24T00:59:15,761230613+00:00 - rsyslog/run 
2020-12-24T00:59:15,761980212+00:00 - gunicorn/run 
2020-12-24T00:59:15,764719608+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_17c0275e5cc0743395fc22f932d02cb3/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_17c0275e5cc0743395fc22f932d02cb3/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_17c0275e5cc0743395fc22f932d02cb3/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_17c0275e5cc0743395fc22f932d02cb3/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_17c0275e5cc0743395fc22f932d02cb3/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-12-24T00:59

WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

In [39]:
scoring_uri = 'scoring uri for your service'
headers = {'Content-Type':'application/json'}

test_data = json.dumps({'data':[{
    'age':75,
    'anaemia':0,
    'creatinine_phosphokinase':582,
    'diabetes':0,
    'ejection_fraction':20,
    'high_blood_pressure':1,
    'platelets':265000,
    'serum_creatinine':1.9,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':4}
    ]
        })

response = requests.post(scoring_uri, data=test_data, headers=headers)

print("Result:",response.text)

MissingSchema: Invalid URL 'scoring uri for your service': No schema supplied. Perhaps you meant http://scoring uri for your service?

In [ ]:
env.save_to_directory('./training',overwrite=False)

print(service.get_logs())



TODO: In the cell below, print the logs of the web service and delete the service

In [26]:
print(response.status_code)
print(response.elapsed)
print(response.json())

service.delete()

NameError: name 'response' is not defined